In [1]:
import pandas as pd

In [2]:
df = pd.read_parquet('parquet_output/20.parquet')
df.count()

steam_appid             8688
required_age            8688
is_free                 8688
controller_support         0
detailed_description    8688
about_the_game          8688
short_description       8688
price_overview          8688
metacritic_score           0
categories              8688
genres                  8688
recommendations         8688
achievements               0
release_coming_soon     8688
release_date            8688
review_language         8688
review                  8688
voted_up                8688
votes_up                8688
votes_funny             8688
weighted_vote_score     8688
dtype: int64

In [ ]:
columns = df.columns.tolist()
unique_ids = df['steam_appid'].unique()
print(f"Found {len(unique_ids)} unique Steam app IDs")
total_counts = 0
# Count occurrences of each app ID
for app_id in unique_ids:
    count = len(df[df['steam_appid'] == app_id])
    print(f"App ID {app_id}: {count} reviews")

# Alternative method using value_counts()
print("\nUsing value_counts() method:")
app_counts = df['steam_appid'].value_counts().sort_index()
for app_id, count in app_counts.items():
    print(f"App ID {app_id}: {count} reviews")
    total_counts += count


Found 59 unique Steam app IDs
App ID 10: 117100 reviews
App ID 30: 7064 reviews
App ID 80: 30936 reviews
App ID 3310: 1246 reviews
App ID 3600: 293 reviews
App ID 40340: 205 reviews
App ID 43500: 185 reviews
App ID 208520: 2331 reviews
App ID 216210: 181 reviews
App ID 216890: 1766 reviews
App ID 217690: 1595 reviews
App ID 227000: 3282 reviews
App ID 227020: 501 reviews
App ID 239820: 21700 reviews
App ID 245620: 14088 reviews
App ID 246760: 1788 reviews
App ID 257220: 212 reviews
App ID 261470: 1813 reviews
App ID 270830: 128 reviews
App ID 279990: 267 reviews
App ID 281410: 570 reviews
App ID 287070: 343 reviews
App ID 297750: 474 reviews
App ID 297760: 415 reviews
App ID 304350: 542 reviews
App ID 306370: 114 reviews
App ID 306630: 330 reviews
App ID 306640: 225 reviews
App ID 310390: 163 reviews
App ID 312350: 282 reviews
App ID 312360: 187 reviews
App ID 312370: 985 reviews
App ID 319850: 555 reviews
App ID 330420: 269 reviews
App ID 330720: 249 reviews
App ID 337680: 158 reviews